In [ ]:
### INSTALL CASADI ####

!pip install casadi

In [2]:
import casadi as ca
import numpy as np
import time
import math
import scipy.io

v_e = 0.75
v_p = 0.75
N = 20
dt = 1/N


opti = ca.Opti()

# constraint lambda, mu
lam = opti.variable(3, N - 1)  # index = 0,...,T-2
mu = opti.variable(1, N)
mu_bar = opti.variable(1, N)
epsilon = opti.variable(1, N)
C = opti.variable(1, 1)
action = opti.variable(1, 1)
gamma = opti.variable(1, N)

# control variable
u = opti.variable(1, N)
d = opti.variable(1, N)


# state
states = opti.variable(3, N)
x1 = states[0, :]
x2 = states[1, :]
x3 = states[2, :]

# parameters
opt_x0 = opti.parameter(3)

# dynamic function
def f(x_, u_, d_): return ca.vertcat(
     -0.75 + 0.75 * ca.cos(x_[2]) + u_ * x_[1],
      0.75 * ca.sin(x_[2]) - u_ * x_[0],
      d_ - u_)

def l(x_): return ca.sqrt(x_[0]**2 + x_[1]**2 + 1e-9) - 0.25

# gradient l(x_i) w.r.t x_i
def dldx(x_): return ca.vertcat(
      x_[0]/ca.sqrt(x_[0]**2 + x_[1]**2 + 1e-9),
      x_[1]/ca.sqrt(x_[0]**2 + x_[1]**2 + 1e-9),
      0)

# gradient f(x_i) w.r.t x_i, I + df/dx * dt
def dfdx(x_, u_, dt):
    row1 = ca.horzcat(1, -u_*dt, 0)
    row2 = ca.horzcat(u_*dt, 1, 0)
    row3 = ca.horzcat(-0.75*ca.sin(x_[2])*dt, 0.75*ca.cos(x_[2])*dt, 1)
    return ca.vertcat(row1,
                      row2,
                      row3)

def u_star(lam, x_, dt):     
    return 3*ca.sign((lam[0]*x_[1] - lam[1]*x_[0]-lam[2])*dt)
    
def d_star(lam, x_, dt):return 3*ca.sign(lam[2]*dt)
    

# init_condition
opti.subject_to(states[:, 0] == opt_x0)

# dynamic constraint
for i in range(N - 1):
#     u[:, i] = u_star(lam[:, i], states[:, i], dt)
    d[:, i] = d_star(lam[:, i], states[:, i], dt)
    x_next = states[:, i] + f(states[:, i], u[:, i], d[:, i]) * dt
    opti.subject_to(states[:, i + 1] == x_next)
    

# costate constraint
for i in range(1, N - 1):
    opti.subject_to(-mu[:, i]*dldx(states[:, i]) +mu_bar[:, i]*dldx(states[:, i]) + lam[:, i - 1] -
                    ca.mtimes(dfdx(states[:, i], u[:, i], dt), lam[:, i]) == 0)
opti.subject_to(-mu[:, N - 1] * dldx(states[:, N - 1]) + mu_bar[:, N - 1] * dldx(states[:, N - 1])+
                lam[:, N - 2] == 0)

# multiplier constraint
sum_mu = 0
sum_mu2 = 0
for i in range(N):
    sum_mu = sum_mu + mu[:, i] - mu_bar[:, i]
    sum_mu2 = sum_mu - mu_bar[:, i] - gamma[:, i]
    opti.subject_to(mu[:, i] * (C - l(states[:, i])) == 0)
    opti.subject_to(mu_bar[:, i] * (-C + l(states[:, i]) - epsilon[:, i]) == 0)
    opti.subject_to(epsilon[:, i] * gamma[:, i] == 0)
    opti.subject_to(epsilon[:, i] >= 0)
    opti.subject_to(mu[:, i] >= 0)
    opti.subject_to(mu_bar[:, i] >= 0)
    opti.subject_to(gamma[:, i] >= 0)
    opti.subject_to(C <= l(states[:, i]))
    opti.subject_to(C >= l(states[:, i]) - epsilon[:, i])
opti.subject_to(sum_mu + 1 == 0)
opti.subject_to(sum_mu2 + 1 == 0)


### INITIAL GUESSES load from the matrix data_timeSteps --> 20 means 20 time steps | (20, 50, 100) choices
data = scipy.io.loadmat('ground_truth_data_20.mat')
x1_gus = data['x1'].flatten().tolist()
x2_gus = data['x2'].flatten().tolist()
x3_gus = data['x3'].flatten().tolist()

opti.set_initial(x1, x1_gus)
opti.set_initial(x2, x2_gus)
opti.set_initial(x3, x3_gus)

# objective function
obj = -C
# for i in range(N):
#     obj = obj + epsilon[:, i]

opti.minimize(obj)

# boundrary and control conditions
opti.subject_to(opti.bounded(-1.0, x1, 1.0))
opti.subject_to(opti.bounded(-1.0, x2, 1.0))
opti.subject_to(opti.bounded(-math.pi, x3, math.pi))

opti.subject_to(opti.bounded(-3, u, 3))

ini_state = np.array([[0.5], [-0.5], [1.59]])
opti.set_value(opt_x0, ini_state)

opts_setting = {'ipopt.hessian_approximation':"limited-memory", 'ipopt.max_iter': 15000, 'ipopt.print_level': 5, 'print_time': 0,
                'ipopt.acceptable_tol': 1e-8, 'ipopt.acceptable_obj_change_tol': 1e-6, 'ipopt.inf_pr_output':"original"}

opti.solver('ipopt', opts_setting)
sol = opti.solve()

res_x = sol.value(states)
res_u = sol.value(u)
res_d = sol.value(d)
res_lambda = sol.value(lam)
res_mu = sol.value(mu)
res_epsilon = sol.value(epsilon)

print(res_x)

print()




This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     1033
Number of nonzeros in inequality constraint Jacobian.:      300
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      238
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      179
Total number of inequality constraints...............:      200
        inequality constraints with only lower bounds:       80
   inequality constraints with lower and upper bounds:       80
        inequality constraints with only upper bounds:       40

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

 157r 4.2868153e-01 5.46e-01 8.84e+02   0.3 2.31e+00    -  1.27e-01 4.21e-03h  8
 158r 3.9075245e-01 4.76e-01 7.95e+02   0.3 2.61e+00    -  2.36e-01 1.27e-01h  2
 159r 3.7737480e-01 4.25e-01 3.99e+02   0.3 1.15e+00    -  9.77e-01 1.10e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 160r 4.4386001e-01 3.00e-01 4.03e+02   0.3 4.74e+00    -  1.56e-01 2.57e-01f  1
 161r 3.9331943e-01 6.49e-01 6.91e+02   0.3 4.85e+00    -  5.71e-01 5.99e-01F  1
 162r 4.5566388e-01 6.49e-01 7.90e+02   0.3 4.25e+00    -  8.89e-01 2.75e-01F  1
 163r 6.4178730e-01 8.86e-01 4.66e+02   0.3 2.03e+00    -  5.59e-01 5.49e-01f  1
 164r 5.5190511e-01 7.65e-01 4.85e+02   0.3 2.84e+00    -  8.28e-01 1.25e-01f  4
 165r 4.1559217e-01 5.66e-01 3.43e+02   0.3 2.06e+00    -  1.00e+00 2.50e-01h  3
 166r 5.3900702e-01 3.48e-01 2.62e+02   0.3 6.16e+00    -  4.16e-01 2.36e-01f  2
 167r 5.8537975e-01 4.17e-01 7.53e+02   0.3 4.79e+00    -  8.65e-01 1.01e-01h  3
 168r 5.9113577e-01 4.33e-01

 272r 2.4736419e-01 4.89e-01 4.40e+02  -6.2 1.97e+00    -  2.41e-01 9.86e-02f  1
 273r 2.7012723e-01 7.95e-01 2.03e+02  -0.7 1.64e+00    -  7.70e-01 3.75e-01f  1
 274r 4.8395213e-01 1.99e+00 2.77e+03   0.7 8.74e+00    -  2.77e-02 4.05e-01f  1
 275r 4.2215351e-01 5.10e-01 1.23e+03   0.2 8.41e+00    -  3.17e-01 6.65e-01f  1
 276r 4.4253051e-01 5.19e-01 1.22e+03   0.2 4.63e+00    -  8.60e-01 1.87e-01f  1
 277r 2.9390483e-01 3.00e-01 6.71e+02   0.2 2.78e+00    -  4.04e-01 3.78e-01h  1
 278r 3.0031754e-01 2.83e-01 3.89e+02   0.2 2.85e+00    -  7.32e-02 3.56e-01h  1
 279r 2.8052391e-01 2.43e-01 3.75e+02   0.2 3.32e+00    -  7.14e-02 1.42e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 280r 2.7788441e-01 2.18e-01 3.07e+02   0.2 2.32e+00    -  1.47e-01 1.03e-01f  1
 281r 2.6358243e-01 2.00e-01 3.82e+02   0.2 2.09e+00    -  2.60e-01 8.23e-02f  1
 282  2.8069872e-01 1.97e-01 7.23e+02   0.3 3.95e+00    -  1.00e+00 1.51e-02h  1
 283  2.0879858e-01 2.40e-01

 422  2.5424680e-01 3.90e-01 9.00e+03   0.2 2.71e+01    -  1.17e-01 1.59e-02f  4
 423  2.5035644e-01 3.93e-01 1.10e+04   0.2 3.63e+01    -  8.45e-02 1.50e-02f  3
 424  2.5452623e-01 3.96e-01 9.15e+03   0.2 3.68e+01    -  7.79e-02 1.80e-02h  3
 425  2.5358578e-01 3.94e-01 1.82e+04   0.2 2.57e+01    -  4.42e-01 5.74e-03h  5
 426  2.5470242e-01 3.90e-01 1.32e+04   0.2 1.26e+01    -  2.45e-01 1.56e-02h  4
 427  2.5372163e-01 3.88e-01 1.57e+04   0.2 2.22e+01    -  1.44e-01 6.47e-03h  5
 428  2.5689107e-01 3.78e-01 1.47e+04   0.2 6.00e+00    -  9.38e-01 7.88e-02h  2
 429  2.5402394e-01 3.93e-01 1.29e+04   0.2 6.10e+01    -  3.36e-02 9.33e-03f  3
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 430  2.5588542e-01 3.89e-01 1.34e+04   0.2 5.43e+01    -  5.58e-02 2.51e-02h  1
 431  2.5758650e-01 3.92e-01 1.77e+04   0.2 1.72e+01    -  1.49e-01 2.53e-02f  3
 432  2.5606683e-01 4.01e-01 1.45e+04   0.2 4.78e+01    -  2.81e-02 4.12e-03f  4
 433  2.5666462e-01 4.01e-01

 553  2.4696361e-01 1.92e-01 1.08e+14   0.6 2.47e+02    -  4.11e-02 1.02e-03h  4
 554  2.4736598e-01 1.91e-01 1.76e+14   0.6 3.81e+02    -  9.58e-03 4.76e-04h  5
 555  2.4777682e-01 1.91e-01 6.84e+14   0.6 6.15e+02    -  1.86e-02 2.99e-04h  5
 556  2.4816429e-01 1.89e-01 6.83e+14   0.6 8.04e+02    -  1.41e-03 1.67e-03h  1
 557  2.5064091e-01 1.86e-01 6.73e+14   0.6 1.02e+02    -  1.59e-02 1.14e-02h  2
 558  2.5180933e-01 1.86e-01 6.65e+14   0.6 2.40e+02    -  1.11e-02 2.76e-03h  2
 559  2.5394548e-01 1.85e-01 6.60e+14   0.6 3.52e+02    -  7.39e-03 4.77e-03h  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 560  2.5235485e-01 1.78e-01 6.30e+14   0.6 1.09e+02    -  4.44e-02 3.91e-02h  1
 561  2.5399016e-01 1.73e-01 6.30e+14   0.6 1.19e+02    -  8.83e-04 2.48e-02f  2
 562  2.5332915e-01 1.73e-01 6.22e+14   0.6 3.47e+02    -  1.26e-02 4.26e-03h  2
 563  2.5392323e-01 1.72e-01 6.16e+14   0.6 3.43e+02    -  9.47e-03 3.24e-03f  3
 564  2.5061811e-01 1.69e-01

...

14504r 5.3862673e+00 4.77e-01 5.68e+01  -1.8 2.59e-01    -  2.16e-01 3.88e-03h  8
14505r 5.3862674e+00 4.77e-01 5.90e+01  -1.8 2.57e-01    -  4.19e-01 1.93e-03h 10
14506r 5.3862675e+00 4.77e-01 5.81e+01  -1.8 2.52e-01    -  2.36e-01 1.11e-03h 10
14507r 5.3862675e+00 4.77e-01 6.02e+01  -1.8 2.53e-01    -  4.14e-01 3.02e-05h 16
14508r 5.3862675e+00 4.77e-01 5.91e+01  -1.8 2.51e-01    -  2.38e-01 1.79e-05h 16
14509r 5.3862913e+00 4.85e-01 3.51e+01  -1.8 2.53e-01    -  4.06e-01 4.06e-01s 22
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
14510r 5.3862856e+00 4.79e-01 3.26e+01  -1.8 8.51e-01    -  7.14e-02 7.14e-02s 22
14511r 5.3863286e+00 4.86e-01 4.42e+00  -1.8 1.97e-01    -  9.40e-01 9.40e-01s 22
14512r 5.3863291e+00 4.85e-01 3.97e+00  -1.8 2.94e-01    -  2.99e-02 2.99e-02s 22
14513r 5.3863320e+00 4.82e-01 2.39e+00  -1.8 2.90e-01    -  1.76e-01 1.76e-01s 22
14514r 5.3863320e+00 4.82e-01 9.78e-01  -1.8 2.95e-01    -  0.00e+00 0.00e+00R  1
14515r 5.3863294e

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
14640r 5.2615077e+00 4.86e-01 2.94e+00  -2.1 3.83e-01    -  1.00e+00 1.57e-01f  3
14641r 5.2610408e+00 4.76e-01 3.94e+00  -2.1 5.77e-01    -  2.74e-01 2.50e-01h  3
14642r 5.2607260e+00 4.82e-01 8.83e+00  -2.1 4.67e-01    -  1.00e+00 1.74e-01f  3
14643r 5.2604403e+00 4.86e-01 2.52e+00  -2.1 3.86e-01    -  1.00e+00 1.59e-01f  3
14644r 5.2585987e+00 4.44e-01 2.04e+01  -2.1 5.75e-01    -  2.80e-01 1.00e+00w  1
14645r 5.2568374e+00 5.17e-01 3.10e+01  -2.1 9.00e-01    -  1.00e+00 9.91e-01w  1
14646r 5.2568299e+00 5.16e-01 3.07e+01  -2.1 3.51e+00    -  1.51e-03 3.87e-03w  1
14647r 5.2599799e+00 4.76e-01 3.73e+00  -2.1 1.48e+00    -  2.80e-01 2.50e-01h  2
14648r 5.2596674e+00 4.82e-01 8.67e+00  -2.1 4.68e-01    -  1.00e+00 1.75e-01f  3
14649r 5.2593834e+00 4.86e-01 2.28e+00  -2.1 3.86e-01    -  1.00e+00 1.60e-01f  3
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
14650r 5.2589294e+

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
14760r 5.6998818e+00 4.81e-01 1.02e+02  -1.1 3.98e+00    -  4.18e-01 3.70e-01f  1
14761r 5.7073794e+00 4.59e-01 8.37e+01  -1.1 8.91e+00    -  1.48e-01 2.31e-01h  1
14762r 5.7088471e+00 4.61e-01 9.63e+01  -1.1 1.07e+00    -  1.42e-01 4.52e-02f  1
14763r 5.7218856e+00 4.64e-01 1.91e+02  -1.1 4.64e+00    -  5.64e-02 4.05e-01F  1
14764r 5.7231220e+00 4.65e-01 1.92e+02  -1.1 8.02e-01    -  1.00e+00 3.96e-02h  1
14765r 5.7322711e+00 4.77e-01 1.35e+02  -1.1 4.83e+00    -  2.37e-01 2.93e-01f  1
14766r 5.7384323e+00 4.67e-01 1.08e+02  -1.1 5.18e+00    -  1.84e-01 2.05e-01h  2
14767r 5.7391712e+00 4.68e-01 1.08e+02  -1.1 1.50e+00    -  1.00e+00 2.47e-02h  1
14768r 5.7536843e+00 4.86e-01 5.34e+01  -1.1 4.45e+00    -  3.44e-01 4.86e-01f  2
14769r 5.7575318e+00 4.71e-01 4.82e+01  -1.1 9.14e+00    -  9.21e-02 1.38e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
14770r 5.7858054e+

14891r 5.6379294e+00 3.03e-01 1.97e+01  -1.8 8.63e-01    -  7.62e-01 9.97e-01H  1
14892r 5.6333080e+00 4.05e-01 1.25e+01  -1.8 1.50e+00    -  5.36e-01 4.95e-01h  2
14893r 5.6243411e+00 3.76e-01 1.40e+01  -1.8 1.19e+00    -  1.00e+00 9.92e-01H  1
14894r 5.6166527e+00 4.99e-01 3.38e+01  -1.8 1.20e+00    -  9.21e-01 8.72e-01h  1
14895r 5.6145295e+00 4.87e-01 2.22e+01  -1.8 1.39e+00    -  3.50e-02 2.49e-01F  1
14896r 5.6123314e+00 4.92e-01 3.97e+01  -1.8 1.25e+00    -  1.00e+00 2.62e-01f  2
14897r 5.6104875e+00 4.95e-01 3.62e+01  -1.8 2.50e-01    -  1.00e+00 2.22e-01f  2
14898r 5.6063159e+00 4.70e-01 1.57e+01  -1.8 7.15e-01    -  7.55e-02 5.00e-01f  2
14899r 5.6062396e+00 4.69e-01 3.10e+01  -1.8 5.47e-01    -  3.48e-03 9.18e-03F  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
14900r 5.6001678e+00 5.01e-01 1.34e+01  -1.8 5.55e-01    -  1.00e+00 7.47e-01f  1
14901r 5.5999800e+00 4.97e-01 5.04e+01  -1.8 1.34e+00    -  3.81e-03 2.30e-02h  1
14902r 5.5945206e

RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:159:
.../casadi/core/optistack_internal.cpp:999: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Maximum_Iterations_Exceeded'

In [ ]:
opti.debug.value(d)

In [ ]:
sol.value(l)